In [43]:
LAYER_NAME = 'layer_1a'

import tensorflow as tf 
import tensorflow_io as tfio
import tensorflow_probability as tfp

print(tf.__version__)
print('Listing all GPU resources:')
print(tf.config.experimental.list_physical_devices('GPU'))
print()
import tensorflow.keras as keras
print(tfp.__version__)
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import pickle
import os
from tqdm import trange
import sys
import git
import importlib

from file_path_dict import file_path_dict

2.2.0
Listing all GPU resources:
[]

0.9.0


In [44]:
LAYER_NAME = 'encoder_1a'

FILTERS = 32
DATA_SIZE = 60000
PRIOR_MU = 0
PRIOR_SIGMA = 10

BATCH_SIZE = 128
EPOCHS = 200
VERBOSE = 2

ROOT_PATH = git.Repo("", search_parent_directories=True).git.rev_parse("--show-toplevel")
DATA_PATH = ROOT_PATH + "/data/"
SAVE_PATH = ROOT_PATH + "/" + LAYER_NAME + "/" + LAYER_NAME + "_bayesian_model.h5"
PICKLE_PATH = ROOT_PATH + "/" + LAYER_NAME + "/" + LAYER_NAME + '_hist.pkl'
MODEL_PATH = ROOT_PATH + "/" + LAYER_NAME + "/" + LAYER_NAME + "_model"

In [45]:
print("-" * 30)
print("Constructing model...")
print("-" * 30)
spec = importlib.util.spec_from_file_location(MODEL_PATH, MODEL_PATH + ".py")
ModelLoader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ModelLoader)
model = ModelLoader.make_model()
print(model.summary())

------------------------------
Constructing model...
------------------------------

Input size: (None, 144, 144, 4)
Output size: (None, 144, 144, 1)
Model: "model_encoder_1a"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        [(None, 144, 144, 4) 0                                            
__________________________________________________________________________________________________
encoder_1_a (Conv2DFlipout)     (None, 144, 144, 32) 4736        input_layer[0][0]                
__________________________________________________________________________________________________
encoder_1_b (Conv2D)            (None, 144, 144, 32) 9248        encoder_1_a[0][0]                
__________________________________________________________________________________________________
downsample_1 (MaxPooling2D)     

In [41]:
model.load_weights(SAVE_PATH)
print("Model weights loaded successfully\n")

Model weights loaded successfully



In [42]:
n_test = 200
msks_test = np.load('../data/msks_test.npy')[:n_test]
imgs_test = np.load('../data/imgs_test.npy')[:n_test]

print("First " + str(n_test) + " test samples loaded\n")

First 200 test samples loaded



In [ ]:
Xy_test = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(imgs_test),
                                tf.data.Dataset.from_tensor_slices(msks_test))
                             ).cache().batch(BATCH_SIZE).prefetch(8)

In [ ]:
prediction_size = list(msks_test.shape)
prediction_size.insert(0, N_PREDICTIONS)
prediction_test = np.zeros(prediction_size)

In [ ]:
print("Getting Monte Carlo samples of test predictions...")
for i in trange(N_PREDICTIONS):
    prediction_test[i] = model.predict(Xy_test)

In [ ]:
for i in trange(100, 101):
    plt.figure(dpi=200)
    plt.subplot(221)
    plt.title('Test Data 1')
    plt.imshow(imgs_test[i, :, :, 0])

    plt.subplot(222)
    plt.title('Test Data 2')
    plt.imshow(imgs_test[i, :, :, 1])

    plt.subplot(223)
    plt.title('Test Data 3')
    plt.imshow(imgs_test[i, :, :, 2])

    plt.subplot(224)
    plt.title('Test Data 4')
    plt.imshow(imgs_test[i, :, :, 3])
    plt.tight_layout()
    plt.savefig('../images/' + LAYER_NAME + '/input_images_'+str(i).zfill(4)+'.png')
    plt.close()


    plt.figure(dpi=200)

    plt.subplot(221)
    plt.title('\nPredicted Label Mean')
    plt.imshow(prediction_test.mean(0)[i, :, :, 0])
    plt.clim(0, 1)

    plt.subplot(222)
    plt.title('\nPredicted Label Stddev')
    plt.imshow(prediction_test.std(0)[i, :, :, 0])
    plt.clim(0, 0.2)

    plt.subplot(223)
    plt.title('True Label')
    plt.imshow(msks_test[i, :, :, 0])
    plt.clim(0, 1)
    plt.suptitle('Test Image: ' + str(i) + '\n')

    plt.tight_layout()

    plt.savefig('../images/' + LAYER_NAME + '/prediction_images_'+str(i).zfill(4)+'.png')


    plt.figure(dpi=200)

    plt.subplot(221)
    plt.title('\nPredicted Label 50%')
    plt.imshow(prediction_test.mean(0)[i, :, :, 0])
    plt.clim(0, 1)

    plt.subplot(222)
    plt.title('\nPredicted Label 12%')
    plt.imshow(np.clip(prediction_test.mean(0)[i, :, :, 0] - 2*prediction_test.std(0)[i, :, :, 0], 0, 1))
    plt.clim(0, 1)

    plt.subplot(223)
    plt.title('Predicted Label 98%')
    plt.imshow(np.clip(prediction_test.mean(0)[i, :, :, 0] + 2*prediction_test.std(0)[i, :, :, 0], 0, 1))
    plt.clim(0, 1)

    plt.subplot(224)
    plt.title('True Label')
    plt.imshow(msks_test[i, :, :, 0])
    plt.clim(0, 1)

    plt.suptitle('Test Image: ' + str(i) + '\n')

    plt.tight_layout()
    plt.savefig('../images/' + LAYER_NAME + '/prediction_percentile_images_'+str(i).zfill(4)+'.png')

In [10]:
history = pickle.load(open(PICKLE_PATH, 'rb'))

In [ ]:
plt.figure(dpi=400)
plt.semilogy(history['loss'], label='training loss (ELBO)')
plt.semilogy(history['val_loss'], label='testing loss')
plt.legend()

In [ ]:
for layer in model.layers:
    print(layer)
    weights = layer.get_weights()
    for w in weights:
        print(w.shape)